In [2]:
import datetime
import os
import pickle
import glob
from pathlib import Path

import cv2
import faiss
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import  davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import MinMaxScaler


In [3]:
source_path = "E:/taha/code/CanSat/images"

images = []

frames = Path(source_path)
for path in frames.glob('*.jpg'):

    image = cv2.imread(str(path))
    images.append(image)

shape = images[0].shape    

In [4]:
Size = len(images)

images = np.asarray(images)
images = images.reshape(images.shape[0], -1)

In [5]:
# images.shape
shape

(3456, 5184, 3)

In [6]:
row_indices = np.arange(0, shape[1])
column_indices = np.arange(0, shape[0])

row_mesh, column_mesh = np.meshgrid(row_indices, column_indices)

index_array = np.stack([column_mesh, row_mesh], axis=-1)

hsv_image = np.zeros((images.shape[0], shape[0], shape[1], 5))

images = images.reshape(Size, shape[0], shape[1], shape[2])
for i in range(Size):
    temp = cv2.cvtColor(images[i], cv2.COLOR_BGR2HSV)
    hsv_image[i] = np.concatenate((temp, index_array), axis=2)

images = images.reshape(Size, shape[0] * shape[1] * shape[2])

In [7]:
hsv_image.reshape(Size, shape[0] * shape[1], 5)[0]

array([[1.000e+01, 7.900e+01, 1.070e+02, 0.000e+00, 0.000e+00],
       [1.000e+01, 7.700e+01, 1.090e+02, 0.000e+00, 1.000e+00],
       [1.000e+01, 7.900e+01, 1.070e+02, 0.000e+00, 2.000e+00],
       ...,
       [1.100e+01, 6.600e+01, 1.350e+02, 3.455e+03, 5.181e+03],
       [1.100e+01, 6.500e+01, 1.380e+02, 3.455e+03, 5.182e+03],
       [1.100e+01, 6.500e+01, 1.370e+02, 3.455e+03, 5.183e+03]])

In [8]:
images_shape = hsv_image.shape

In [9]:
def set_priority(arr, color, pos):
    color_columns = np.tile(arr[:, :3], (1, color))
    pos_columns = np.tile(arr[:, 3:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [10]:
hsv_image = hsv_image.reshape(images_shape[0] * images_shape[1] * images_shape[2], images_shape[3])

minMaxScaler = MinMaxScaler()
minMaxScaler.fit(hsv_image)

hsv_image = hsv_image.reshape(images_shape[0], images_shape[1] * images_shape[2], images_shape[3])

In [11]:
k1 = 15
color = 1
pos = 5

In [12]:
rgb_codes = [
    (255, 0, 0),  # Red
    (0, 255, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Yellow
    (0, 255, 255),  # Cyan
    (255, 0, 255),  # Magenta
    (255, 165, 0),  # Orange
    (128, 0, 128),  # Purple
    (0, 128, 128),  # Teal
    (255, 192, 203),  # Pink
    (190, 255, 0),  # Lime
    (165, 42, 42),  # Brown
    (255, 215, 0),  # Gold
    (112, 128, 144),  # Slate Gray
    (128, 128, 0)  # Olive
]

# target color to look for
target_color = np.array([0,0,255])

In [13]:
if not os.path.isdir(f'colors'):
    os.mkdir(f'colors')
if not os.path.isdir(f'colors/k{k1}_color{color}_pos{pos}'):
    os.mkdir(f'colors/k{k1}_color{color}_pos{pos}')
else:
    if os.path.exists(f'colors/k{k1}_color{color}_pos{pos}/res.csv'):
        os.remove(f'colors/k{k1}_color{color}_pos{pos}/res.csv')

list_of_all_clusters = []

for image in range(images_shape[0]):
    print(f'image {image} {datetime.datetime.now()}')

    hsv_image_scaled = minMaxScaler.transform(hsv_image[image])
    hsv_image_scaled = set_priority(hsv_image_scaled, color, pos)

    kmeans = faiss.Kmeans(d=hsv_image_scaled.shape[1], k=k1, nredo=4) # d =(color*3 + pos*2)
    kmeans.train(hsv_image_scaled)
    # print(f"d = {hsv_image.shape}")

    list_of_all_clusters.append(kmeans.assign(hsv_image_scaled)[1])

    clusters = [[] for i in range(k1)]
    for index, value in enumerate(list_of_all_clusters[-1]):
        clusters[value].append(hsv_image[image, index])

    labels_for_segment_image = list_of_all_clusters[-1].reshape(shape[0], shape[1])
    # print(f"labels shape = {labels_for_segment_image.shape}")
    centers = kmeans.centroids

    segmented_image = np.zeros((shape[0], shape[1], 3), dtype=np.float32)
    for i in range(k1):
        segmented_image[labels_for_segment_image == i] = np.array(rgb_codes[i]) / 255
        # print(f"segmented_image shape = {labels_for_segment_image == i}")

    plt.imshow(segmented_image)
    plt.savefig(f'colors/k{k1}_color{color}_pos{pos}/RGB{image}.jpg')
    plt.close()

    segmented_image = np.zeros((shape[0], shape[1], 3), dtype=np.float32)
    for i in range(k1):
        segmented_image[labels_for_segment_image == i] = centers[i][0]
        # print(f"centers = {centers[i][8]}")

    plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_HSV2BGR))
    plt.savefig(f'colors/k{k1}_color{color}_pos{pos}/HSV{image}.jpg')
    plt.close()

    # color check
    min_distance = np.inf
    target_k = -1
    original_image = images[image].reshape(shape)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    segmented_image = np.zeros((shape[0], shape[1], 3), dtype=np.float32)
    for i in range(k1):
        segmented_image[labels_for_segment_image == i] = original_image[labels_for_segment_image == i] / 255
        distance = np.linalg.norm(segmented_image - (target_color/255))
        if distance < min_distance:
            min_distance = distance
            target_k = i
        # print(f"tk = {target_k}, d = {distance}")
        segmented_image = np.zeros((shape[0], shape[1], 3), dtype=np.float32)
    segmented_image[labels_for_segment_image == target_k] = original_image[labels_for_segment_image == target_k] / 255
    print(np.count_nonzero(segmented_image))

    np.savez_compressed(f'colors/k{k1}_color{color}_pos{pos}/target{image}.npz',segmented_image=segmented_image)

    plt.imshow(segmented_image)
    # plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
    plt.savefig(f'colors/k{k1}_color{color}_pos{pos}/target{image}.jpg')
    plt.close()


image 0 2024-05-20 20:07:12.533471
2783107
image 1 2024-05-20 20:07:45.592612
2877996
image 2 2024-05-20 20:08:19.628537
2747468


## area

In [14]:
for image in range(images_shape[0]):
    print(f'image {image} {datetime.datetime.now()}')
    
    # npz = np.load(f'colors/k{k1}_color{color}_pos{pos}/target{image}.npz')
    # segmented_image = npz["segmented_image"]
    print(f'd = {segmented_image.shape}')
    pixs = np.count_nonzero(segmented_image)

    H = 200
    f = 0.00474 # 4.74 mm

    Area_real = pixs*H*H/f*f

    print(f"pixels = {pixs}, Area = {Area_real}")

image 0 2024-05-20 20:08:52.493177
d = (3456, 5184, 3)
pixels = 2747468, Area = 109898719999.99998
image 1 2024-05-20 20:08:52.643373
d = (3456, 5184, 3)
pixels = 2747468, Area = 109898719999.99998
image 2 2024-05-20 20:08:52.796897
d = (3456, 5184, 3)
pixels = 2747468, Area = 109898719999.99998


In [30]:
import math

for image in range(images_shape[0]):
    print(f'image {image} {datetime.datetime.now()}')
    
    npz = np.load(f'colors/k{k1}_color{color}_pos{pos}/target{image}.npz')
    segmented_image = npz["segmented_image"]
    # print(f'd = {segmented_image.shape}')
    pixs = np.count_nonzero(segmented_image)

    H = 200 # meter
    # f = 0.00474 # 4.74 mm
    f = 0.035 # 35 mm

    # sensor_size = 0.0025 # 25 mm^2
    # pixel_area = sensor_size/(segmented_image.shape[0]*segmented_image.shape[1])
    # pixel_width = math.sqrt(pixel_area)

    pixel_width = 0.0000014 # 1.4 um
    pixel_area = pixel_width**2
    # print(pixel_area,pixel_size)
    print("pixel area um^2:",(pixel_area*1000000000000))
    print("pixel width in um:",(pixel_width*1000000))

    s = H*H/f*f
    Area_image = pixel_area*pixs
    Area_real = (Area_image)*(s)

    print(f"pixels = {pixs}, Area = {Area_real}")
    print(f'Area_image = {Area_image}, s = {s}')

image 0 2024-05-20 20:40:52.546739
pixel area um^2: 1.9599999999999997
pixel width in um: 1.4
pixels = 2783107, Area = 0.21819558879999998
Area_image = 5.454889719999999e-06, s = 40000.0
image 1 2024-05-20 20:40:53.292070
pixel area um^2: 1.9599999999999997
pixel width in um: 1.4
pixels = 2877996, Area = 0.22563488639999998
Area_image = 5.6408721599999995e-06, s = 40000.0
image 2 2024-05-20 20:40:54.007920
pixel area um^2: 1.9599999999999997
pixel width in um: 1.4
pixels = 2747468, Area = 0.21540149119999996
Area_image = 5.385037279999999e-06, s = 40000.0
